In [1]:
import numpy as np
import cvxpy as cvx

In [2]:
Stages        = 20
initial_state = [1,3,1,0,0]
gradient      = [2,2,2,2,2]
gold_costs    = [1,1,1,1,1,1]
crystal_costs = [1,1,1,1,1,1]

In [3]:
initial_gatherers, initial_houses, initial_workers, initial_crystals, initial_gold = initial_state

In [4]:
# Resources
Crystals   = cvx.Variable(Stages, name = 'Crystals')
Gold       = cvx.Variable(Stages, name = 'Gold')

Raw_Crystal_production = cvx.Variable(Stages, name = 'Crystal Production')
Raw_Gold_production    = cvx.Variable(Stages, name = 'Gold Production')

In [5]:
# Peasants
Workers    = cvx.Variable(Stages, name = 'Workers')
Gatherers  = cvx.Variable(Stages, name = 'Gatherers')

In [6]:
# Structures
Houses     = cvx.Variable(Stages, name = 'Houses')
Barracks   = cvx.Variable(Stages, name = 'Gateways')

In [7]:
# Army
Army       = cvx.Variable(Stages, name = 'Army')
Archers    = cvx.Variable(Stages, name = 'Archers')
Knights    = cvx.Variable(Stages, name = 'Knights')

In [8]:
A = cvx.Variable(Stages-1) # Archers
Au = cvx.Variable(Stages-1) # Gold
B = cvx.Variable(Stages-1) # Barracks
C = cvx.Variable(Stages-1) # Crystals
G = cvx.Variable(Stages-1) # Gatherers
H = cvx.Variable(Stages-1) # Houses
K = cvx.Variable(Stages-1) # Knights
W = cvx.Variable(Stages-1) # Workers

In [9]:
army_barracks      = cvx.Parameter()
buildings_workers  = cvx.Parameter()
peasants_houses    = cvx.Parameter()
army_building      = cvx.Parameter()
supplies_houses    = cvx.Parameter()
resources_gatherer = cvx.Parameter()

# Gold
custo_gold_archers   = cvx.Parameter()
custo_gold_barracks  = cvx.Parameter()
custo_gold_gatherers = cvx.Parameter()
custo_gold_houses    = cvx.Parameter()
custo_gold_knights   = cvx.Parameter()
custo_gold_workers   = cvx.Parameter()

# Crystals
custo_crystal_archers   = cvx.Parameter()
custo_crystal_barracks  = cvx.Parameter()
custo_crystal_gatherers = cvx.Parameter()
custo_crystal_houses    = cvx.Parameter()
custo_crystal_knights   = cvx.Parameter()
custo_crystal_workers   = cvx.Parameter()

In [10]:
army_barracks.value, buildings_workers.value, peasants_houses.value, supplies_houses.value, resources_gatherer.value  = gradient

custo_gold_archers.value, custo_gold_barracks.value, custo_gold_gatherers.value, custo_gold_houses.value, custo_gold_knights.value, custo_gold_workers.value = gold_costs

custo_crystal_archers.value, custo_crystal_barracks.value, custo_crystal_gatherers.value, custo_crystal_houses.value, custo_crystal_knights.value, custo_crystal_workers.value = crystal_costs

In [11]:
Constraints = [Archers[0]   == 0,
               Barracks[0]  == 0,
               Gatherers[0] == initial_gatherers,
               Houses[0]    == initial_houses,
               Knights[0]   == 0,
               Workers      == initial_workers,
               Army[0]      == 0,
               Gold[0]      == initial_gold,
               Crystals[0]  == initial_crystals,
               
               
               A  >= 0,
               B  >= 0,
               G  >= 0,
               H  >= 0,
               K  >= 0,
               W  >= 0]

for i in range(Stages-1):
    
    Constraints.append(A[i]   == Archers[i+1]   - Archers[i])
    Constraints.append(Au[i]  == Gold[i+1]      - Gold[i])
    Constraints.append(B[i]   == Barracks[i+1]  - Barracks[i])
    Constraints.append(C[i]   == Crystals[i+1]  - Crystals[i])
    Constraints.append(G[i]   == Gatherers[i+1] - Gatherers[i])
    Constraints.append(H[i]   == Houses[i+1]    - Houses[i])
    Constraints.append(K[i]   == Knights[i+1]   - Knights[i])
    Constraints.append(W[i]   == Workers[i+1]   - Workers[i])
    
    Constraints.append(A[i]  + K[i] <= army_barracks*Barracks[i])
    Constraints.append(B[i]  + H[i] <= buildings_workers*Workers[i])
    Constraints.append(G[i]  + W[i] <= peasants_houses*Houses[i])
    Constraints.append(Raw_Gold_production + 2*Raw_Crystal_production == resources_gatherer*Gatherers[i])

    Constraints.append(Army[i+1] == Archers[i+1] + Knights[i+1])
    Constraints.append(Army[i+1] 
                       + Gatherers[i+1] 
                       + Workers[i+1] <= Houses[i+1]*supplies_houses)

    Constraints.append(Au[i] == Raw_Gold_production
                       - custo_gold_archers*A[i] 
                       - custo_gold_barracks*B[i]
                       - custo_gold_gatherers*G[i]
                       - custo_gold_houses*H[i]
                       - custo_gold_knights*K[i]
                       - custo_gold_workers*W[i])
    
    Constraints.append(C[i] == Raw_Crystal_production
                       - custo_crystal_archers*A[i] 
                       - custo_crystal_barracks*B[i]
                       - custo_crystal_gatherers*G[i]
                       - custo_crystal_houses*H[i]
                       - custo_crystal_knights*K[i]
                       - custo_crystal_workers*W[i])
    
    Constraints.append(Gold[i]  >=  custo_gold_archers*A[i] 
                       + custo_gold_barracks*B[i]
                       + custo_gold_gatherers*G[i]
                       + custo_gold_houses*H[i]
                       + custo_gold_knights*K[i]
                       + custo_gold_workers*W[i])
    
    Constraints.append(Crystals[i]  >=  custo_crystal_archers*A[i] 
                       + custo_crystal_barracks*B[i]
                       + custo_crystal_gatherers*G[i]
                       + custo_crystal_houses*H[i]
                       + custo_crystal_knights*K[i]
                       + custo_crystal_workers*W[i])
    Constraints.append(Raw_Gold_production >= 0)
    Constraints.append(Raw_Crystal_production >= 0)

In [12]:
F_obj = cvx.Maximize(Army[-1])

In [13]:
problem = cvx.Problem(F_obj, Constraints)

In [14]:
problem.solve(solver = 'GUROBI', verbose=True)

Academic license - for non-commercial use only
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 2327 rows, 372 columns and 9148 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 265 rows and 2198 columns
Presolve time: 0.02s
Presolved: 107 rows, 129 columns, 1099 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.0000000e+00   2.000000e+00   1.700002e+07      0s
Extra one simplex iteration after uncrush
      51   -9.1538462e+00   0.000000e+00   0.000000e+00      0s

Solved in 51 iterations and 0.04 seconds
Optimal objective -9.153846154e+00


9.153846153846153

In [15]:
Gatherers.value

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [16]:
Houses.value

array([3.        , 3.        , 3.3974359 , 3.52564103, 3.65384615,
       3.78205128, 3.91025641, 4.03846154, 4.16666667, 4.16666667,
       4.16666667, 4.16666667, 4.23076923, 4.23076923, 4.23076923,
       5.06410256, 5.19230769, 5.19230769, 5.30769231, 5.57692308])

In [17]:
Gold.value

array([0.        , 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.79487179,
       0.92307692, 1.05128205, 1.11538462, 1.24358974, 1.37179487,
       0.66666667, 0.66666667, 0.79487179, 0.80769231, 0.66666667])

In [18]:
Crystals.value

array([0.        , 0.66666667, 0.66666667, 0.66666667, 0.66666667,
       0.66666667, 0.66666667, 0.66666667, 0.66666667, 0.79487179,
       0.92307692, 1.05128205, 1.11538462, 1.24358974, 1.37179487,
       0.66666667, 0.66666667, 0.79487179, 0.80769231, 0.66666667])

In [19]:
Army.value

array([0.        , 0.        , 0.        , 0.53846154, 1.07692308,
       1.61538462, 2.15384615, 2.69230769, 3.23076923, 3.76923077,
       4.30769231, 4.84615385, 5.38461538, 5.92307692, 6.46153846,
       7.        , 7.53846154, 8.07692308, 8.61538462, 9.15384615])

In [20]:
Workers.value

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.])

In [21]:
Houses.value[-1]*2

11.153846153846153

In [22]:
Knights.value

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.53846154, 1.07692308, 1.61538462, 1.61538462, 1.61538462,
       1.61538462, 1.61538462, 1.61538462, 2.15384615, 2.15384615,
       2.15384615, 2.69230769, 3.23076923, 3.76923077, 3.76923077])

In [23]:
Archers.value

array([0.        , 0.        , 0.        , 0.53846154, 1.07692308,
       1.07692308, 1.07692308, 1.07692308, 1.61538462, 2.15384615,
       2.69230769, 3.23076923, 3.76923077, 3.76923077, 4.30769231,
       4.84615385, 4.84615385, 4.84615385, 4.84615385, 5.38461538])